In [2]:
'''Based on https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html'''
'''useful: https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780'''
from torch_geometric.nn import GCNConv
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid

In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
# TODO : Understand the structure of the dataset; it is a graph.
# TODO : Understand GCNConv layer from PyTorch. -> https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html#the-messagepassing-base-class
# https://tkipf.github.io/graph-convolutional-networks/

To add self loops one adds the identity matrix to the *adjacency matrix* **A**:


$
A \to \hat{A} = A + I
$

$
\mathbf{H}^{i+1} = F (\mathbf{H^{i}}, \mathbf{X})
$

Where $F$ is a contraction map&ndash;so the above iterative equation is guaranteed to converge.

a simple example could be:

$
H^{i+1} = \sigma(AH^iW^i)
$

Where $\sigma$ is a non-linear activation function.
A major limitation is that **A** is typically not normalized and therefore the multiplication with **A** will completely change the scale of the feature vectors 

Normalizing **A** such that all rows sum to one, i.e. $D^{−1}A$, where **D** is the diagonal node degree matrix, gets rid of this problem. Multiplying with $D^{−1}A$ now corresponds to taking the average of neighboring node features. In practice, dynamics get more interesting when we use a symmetric normalization, i.e. $D^{−1/2}AD^{−1/2}$

$
D_{ij} =  \delta_{ij}\sum_{k}A_{kj} \implies D^{-1/2}_{ij} = \frac{1}{\sqrt{ \sum_{k}A_{kj}}}\delta_{ij}
$

In [41]:
dataset??

In [42]:
dataset.data??

In [46]:
print(dataset.data.x.shape)
print(dataset.data.edge_index.shape)

torch.Size([2708, 1433])
torch.Size([2, 10556])


In [47]:
GCNConv??

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
    ''' x (Tensor): Node feature matrix. Shape [num_nodes, num_node_features]'''
    ''' edge_index (LongTensor): Graph connectivity in COO format. Shape [2, num_edges]'''
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))